In [ ]:
#perform imports and load dataset
import numpy as np
import pandas as pd
import scipy
import spacy
import re # For regular expressions|||
import string # For handling string
import nltk
import pickle
import sklearn
import vaderSentiment
import matplotlib.pyplot as plt
from nltk.util import ngrams
#import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
#import gensim
nltk.download('punkt')
nltk.download('wordnet')
from matplotlib import pyplot
from numpy import array
#from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy.sparse import csr_matrix

#### Importing and cleaning data

In [ ]:
#uploading data
df = pd.read_csv(r'C:/Users/NB332021/OneDrive - Nedbank/Desktop/Thesis/Data/Model Data/LabData.csv')
df2 = pd.read_excel(r'C:/Users/NB332021/OneDrive - Nedbank/Desktop/Thesis/Data/Model Data/TrainingSet.xlsx')
df1 = df[0:10000]
df.shape

In [ ]:
#isolate only labeled data
df1 = df1.dropna(subset="label").reset_index()
df1["label"].head()

In [ ]:
#CLEANING DATA
#drop duplicates
df1 = df1.drop_duplicates(keep='first')
#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())
#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))
#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))
#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(r"www.\S+",'', x))
#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
#stopwords
nlp = spacy.load('en_core_web_sm')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')
# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True
stop = nlp.Defaults.stop_words
#lemmatisation
# Loading model
nlp = spacy.load('en_core_web_sm')
 
# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()
 
df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))
 
#getting rid of empty text rows
df1.dropna(subset = "text", inplace=True)
df1.shape
 
#removing "lab"
df1['label'] = df1['label'].apply(lambda x: int(0) if x == 'lab' else int(x))
 

#### Feature selection performance check

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#calculating document frequency
#MI doesnt take in continuous data therefore TD-IDf wont work
#need to use document frequency since values are discrete
vectorizer = CountVectorizer()
DF = vectorizer.fit_transform(df1["text"])
#Y = pd.DataFrame(data=X1)
#DF = pd.DataFrame(data=Y, columns=vectorizer.get_feature_names_out())

In [ ]:
#check how many features we are looking at
#might be cutting it off too early
 
#libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
#_____________________________________________________________________________________________
#checking the performance of the feature selection
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
# Fit and transform the training data
X_tfidf = vectorizer.fit_transform(df1['text'])
y = df1["label"]
upper = 4000
jump = 500
lower = 1
#create a loop
XMSE_CHI = [0] * len(range(lower,upper,jump))
XMSE_MI = [0] * len(range(lower,upper,jump))
XMSE_LSA = [0] * len(range(lower,upper,jump))
 
XACC_CHI = [0] * len(range(lower,upper,jump))
XACC_MI = [0] * len(range(lower,upper,jump))
XACC_LSA = [0] * len(range(lower,upper,jump))
 
#RFMSE_CHI = [0] * len(range(lower,upper,jump))
#RFMSE_MI = [0] * len(range(lower,upper,jump))
#RFMSE_LSA = [0] * len(range(lower,upper,jump))
 
#RFACC_CHI = [0] * len(range(lower,upper,jump))
#RFACC_MI = [0] * len(range(lower,upper,jump))
#RFACC_LSA = [0] * len(range(lower,upper,jump))
 
#SMSE_CHI = [0] * len(range(lower,upper,jump))
#SMSE_MI = [0] * len(range(lower,upper,jump))
#SMSE_LSA = [0] * len(range(lower,upper,jump))
 
#SACC_CHI = [0] * len(range(lower,upper,jump))
#SACC_MI = [0] * len(range(lower,upper,jump))
#SACC_LSA = [0] * len(range(lower,upper,jump))
 
for nfeatures in range(lower, upper, jump):
    for n in range(len(range(lower,upper,jump))):
        #need to encode labels
        le = LabelEncoder()
        yX = le.fit_transform(y)
        #transformed -1 -> 0, 0 -> 1, 1 -> 2
        #______________________________________________________________________________________________
        #chi-square
        #approach 1
        chi2_features = SelectKBest(score_func=chi2,k=nfeatures)
        ChiX_selected = chi2_features.fit_transform(X_tfidf, yX)
        # Split the data into training and testing sets
        X_train1, X_test1, y_train1, y_test1 = train_test_split(ChiX_selected, yX, test_size=0.2, random_state=42)
       
        #approach 2
        #information gain
        mi_features = SelectKBest(score_func=mutual_info_classif, k=nfeatures)
        miX_selected = mi_features.fit_transform(DF, yX)
        # Split the data into training and testing sets
        X_train2, X_test2, y_train2, y_test2 = train_test_split(miX_selected, yX, test_size=0.2, random_state=42)
 
        #LSA - PCA (matrix too sparse - LSA similiar to PCA but more suitable for parse matrices
        #perform LSA
        svd = TruncatedSVD(n_components=nfeatures)
        X_svd = svd.fit_transform(X_tfidf)
        # Split the data into training and testing sets
        X_train3, X_test3, y_train3, y_test3 = train_test_split(X_svd, yX, test_size=0.2, random_state=42)
 
        #test the performance of the three feature selection techniques
        #____________________________________________________________________________
       
        #XGBoost
        # Fit the XGBoost model to the training data
        XGB1 = XGBClassifier()
        XGB2 = XGBClassifier()
        XGB3 = XGBClassifier()
        #using chi
        XGB1.fit(X_train1, y_train1)
        #using mi
        XGB2.fit(X_train2, y_train2)
        #using LSA
        XGB3.fit(X_train3, y_train3)
 
        # Predict the labels of the testing data
        y_pred1 = XGB1.predict(X_test1)
        y_pred2 = XGB2.predict(X_test2)
        y_pred3 = XGB3.predict(X_test3)
 
        # Print the accuracy of the model
        XACC_CHI[n]=accuracy_score(y_test1, y_pred1)
        XACC_MI[n]=accuracy_score(y_test2, y_pred2)
        XACC_LSA[n]=accuracy_score(y_test3, y_pred3)
 
        # Print the MSE of the model
        XMSE_CHI[n]=mean_squared_error(y_test1, y_pred1)
        XMSE_MI[n]=mean_squared_error(y_test2, y_pred2)
        XMSE_LSA[n]=mean_squared_error(y_test3, y_pred3)
 
        #random forest
        #____________________________________________________________________________________________
        #Random forest classifier
        # Fit the Random Forest model to the training data
        #RFC1 = RandomForestClassifier()
        #RFC2 = RandomForestClassifier()
        #RFC3 = RandomForestClassifier()
 
        #RFC1.fit(X_train1, y_train1)
        #RFC2.fit(X_train2, y_train2)
        #RFC3.fit(X_train3, y_train3)
 
        # Predict the labels of the testing data
        #y_pred1 = RFC1.predict(X_test1)
        #y_pred2 = RFC2.predict(X_test2)
        #y_pred3 = RFC3.predict(X_test3)
 
        # Print the accuracy of the model
        #RFACC_CHI[n]=accuracy_score(y_test1, y_pred1)
        #RFACC_MI[n]=accuracy_score(y_test2, y_pred2)
        #RFACC_LSA[n]=accuracy_score(y_test3, y_pred3)
 
        # Print the MSE of the model
        #RFMSE_CHI[n]=mean_squared_error(y_test1, y_pred1)
        #RFMSE_MI[n]=mean_squared_error(y_test2, y_pred2)
        #RFMSE_LSA[n]=mean_squared_error(y_test3, y_pred3)
 
        #Support vector machine
        #___________________________________________________________________________________________
        # Fit the Random Forest model to the training data
        #SVC1 = OneVsRestClassifier(SVC())
        #SVC2  = OneVsRestClassifier(SVC())
        #SVC3  = OneVsRestClassifier(SVC())
 
        #SVC1.fit(X_train1, y_train1)
        #SVC2.fit(X_train2, y_train2)
        #SVC3.fit(X_train3, y_train3)
 
        # Predict the labels of the testing data
        #y_pred1 = SVC1.predict(X_test1)
        #y_pred2 = SVC2.predict(X_test2)
        #y_pred3 = SVC3.predict(X_test3)
 
        # Print the accuracy of the model
        #SACC_CHI[n]=accuracy_score(y_test1, y_pred1)
        #SACC_MI[n]=accuracy_score(y_test2, y_pred2)
        #SACC_LSA[n]=accuracy_score(y_test3, y_pred3)
 
        # Print the MSE of the model
        #SMSE_CHI[n]=mean_squared_error(y_test1, y_pred1)
        #SMSE_MI[n]=mean_squared_error(y_test2, y_pred2)
        #SMSE_LSA[n]=mean_squared_error(y_test3, y_pred3)
 
print(XMSE_CHI)
print(XMSE_MI)
print(XMSE_LSA)
 
#print(RFMSE_CHI)
#print(RFMSE_MI)
#print(RFMSE_LSA)
 
#print(SMSE_CHI)
#print(SMSE_MI)
#print(SMSE_LSA)

In [ ]:
#PLOT
#create list
nfeatures = list(range(1,4000,500))
print(nfeatures)

In [ ]:
#plot the comparison of the chi-sqaure feature selection across models
import matplotlib.pyplot as plt
plt.plot(nfeatures, XMSE_CHI)
plt.plot(nfeatures, SMSE_CHI)
plt.plot(nfeatures, RFMSE_CHI)
plt.title("Comparison Chi-square performance across models")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["XGBoost", "SVC", "Random Forest"])
plt.show
 


In [ ]:
#plot the comparison of the IG feature selection across models
plt.plot(nfeatures, XMSE_MI)
plt.plot(nfeatures, SMSE_MI)
plt.plot(nfeatures, RFMSE_MI)
plt.title("Comparison IG performance across models")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["XGBoost", "SVC", "Random Forest"])
plt.show

In [ ]:
#plot the comparison of the LSA feature selection across models
plt.plot(nfeatures, XMSE_LSA)
plt.plot(nfeatures, SMSE_LSA)
plt.plot(nfeatures, RFMSE_LSA)
plt.title("Comparison LSA performance across models")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["XGBoost", "SVC", "Random Forest"])
plt.show

In [ ]:
#Plot all FS techniques performance tested on XGBoost (MSE)
plt.plot(nfeatures, XMSE_CHI)
plt.plot(nfeatures, XMSE_MI)
plt.plot(nfeatures, XMSE_LSA)
plt.title("Comparison of all FS techniques, performance tested by XGBoost")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show

In [ ]:
#Plot all FS techniques performance tested on XGBoost (Accuracy)
plt.plot(nfeatures, XACC_CHI)
plt.plot(nfeatures, XACC_MI)
plt.plot(nfeatures, XACC_LSA)
plt.title("Comparison of all FS techniques, performance tested by XGBoost")
plt.xlabel("Number of features")
plt.ylabel("Accuracy")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show

In [ ]:
#Plot all FS techniques performance tested on Random Forest (MSE)
plt.plot(nfeatures, RFMSE_CHI)
plt.plot(nfeatures, RFMSE_MI)
plt.plot(nfeatures, RFMSE_LSA)
plt.title("Comparison of all FS techniques, performance tested by Random Forest")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show

In [ ]:
#Plot all FS techniques performance tested on Random Forest (Accuracy)
plt.plot(nfeatures, RFACC_CHI)
plt.plot(nfeatures, RFACC_MI)
plt.plot(nfeatures, RFACC_LSA)
plt.title("Comparison of all FS techniques, performance tested by Random Forest")
plt.xlabel("Number of features")
plt.ylabel("Accuracy")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show

In [ ]:
#Plot all FS techniques performance tested on SVM (MSE)
plt.plot(nfeatures, SMSE_CHI)
plt.plot(nfeatures, SMSE_MI)
plt.plot(nfeatures, SMSE_LSA)
plt.title("Comparison of all FS techniques, performance tested by SVC")
plt.xlabel("Number of features")
plt.ylabel("MSE")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show

In [ ]:
#All FS techniques performance tested on SVM (Accuracy)
plt.plot(nfeatures, SACC_CHI)
plt.plot(nfeatures, SACC_MI)
plt.plot(nfeatures, SACC_LSA)
plt.title("Comparison of all FS techniques, performance tested by SVC")
plt.xlabel("Number of features")
plt.ylabel("Accuracy")
plt.legend(["Chi-square", "Mutual information", "LSA"])
plt.show